In [1]:
# Compare:
#  (1) classical quantum phase estimation
#             versus
#  (2) iterative quantum phase estimation
# for estimating pi

In [1]:
! pip install qiskit --quiet
! pip install qiskit-aer --quiet
! pip install qiskit-ibmq-provider --quiet

In [ ]:
# (1) classical quantum phase estimation

In [82]:
import numpy as np
import math
import qiskit.quantum_info as qi

from math import pi
from random import random
from qiskit import *
from qiskit.providers.aer import AerSimulator

In [3]:
simulator = AerSimulator(method = "statevector")
simulator

AerSimulator('aer_simulator')

In [4]:
def qpe_pre(circ_, n_qubits):
    circ_.h(range(n_qubits))
    circ_.x(n_qubits)
    for x in reversed(range(n_qubits)):
        for _ in range(2 ** (n_qubits - 1 - x)):
            circ_.cp(1, n_qubits - 1 - x, n_qubits)

In [5]:
def qft_dagger(circ_, n_qubits):
    for qubit in range(int(n_qubits / 2)):
        circ_.swap(qubit, n_qubits - qubit - 1)
    for j in range(0, n_qubits):
        for m in range(j):
            circ_.cp(-np.pi / float(2 ** (j - m)), m, j)
        circ_.h(j)

In [6]:
def create_circuit(n_qubits):
    circ = QuantumCircuit(n_qubits + 1, n_qubits)
    qpe_pre(circ, n_qubits)
    qft_dagger(circ, n_qubits)
    circ.measure(range(n_qubits), range(n_qubits))
    return circ

In [7]:
def run_job(circ, backend, shots=10000):
    job = execute(circ, backend, shots=shots)
    return job.result().get_counts()

In [8]:
def get_pi_estimate(n_qubits, circ=None, backend=simulator):
    if not circ:
        circ = create_circuit(n_qubits)
    counts = run_job(circ, backend=backend, shots=1000)
    max_counts_result = max(counts, key=counts.get)
    max_counts_result = int(max_counts_result, 2)
    theta = max_counts_result / 2 ** n_qubits
    return (1. / (2 * theta))

In [9]:
nq = list(range(2, 15))
pi_estimates = []
for n in nq:
  the_pi_estimate = get_pi_estimate(n)
  pi_estimates.append(the_pi_estimate)
  print(f"{n} qubits, pi = {the_pi_estimate}")

2 qubits, pi = 2.0
3 qubits, pi = 4.0
4 qubits, pi = 2.6666666666666665
5 qubits, pi = 3.2
6 qubits, pi = 3.2
7 qubits, pi = 3.2
8 qubits, pi = 3.1219512195121952
9 qubits, pi = 3.1604938271604937
10 qubits, pi = 3.1411042944785277
11 qubits, pi = 3.1411042944785277
12 qubits, pi = 3.1411042944785277
13 qubits, pi = 3.1411042944785277
14 qubits, pi = 3.1411042944785277


In [26]:
# (2) iterative quantum phase estimation

In [36]:
q = QuantumRegister(1)
a = QuantumRegister(1)
c = ClassicalRegister(1)

In [81]:
E_1, E_2 = (2 * np.pi * random(), 2 * np.pi * random())

In [74]:
t = 1
unitary = QuantumCircuit(q)
unitary.p(E_2 * t, q[0])
unitary.x(q[0])
unitary.p(E_1 * t, q[0])
unitary.x(q[0])

In [75]:
control_u = unitary.to_gate().control(1)

In [76]:
num_bits_estimate = 10
phase = 0
for k_precision in reversed(range(num_bits_estimate)):
    k_circ = QuantumCircuit(q, a, c)
    k_circ.x(q[0])
    k_circ.h(a[0])
    for order in range(2 ** k_precision):
        k_circ.append(control_u, [0, 1])
    phase_shift = 2 * np.pi * phase * 2 ** k_precision
    k_circ.p(-phase_shift, a[0])
    k_circ.h(a[0])
    k_circ.measure(a[0], c[0])
    job = execute(k_circ, simulator, shots=1000)
    result = job.result()
    counts = result.get_counts()
    value = int(max(counts, key=counts.get))
    phase += value / 2 ** (k_precision + 1)

In [83]:
eigenvalue = 2 * np.pi * phase / t
print("Estimated eigenvalue: {}".format(eigenvalue))

Estimated eigenvalue: 2.9145634969827183
